In [1]:
import os
import numpy as np

from utilities import *
from train_model import *

# Load Data

In [2]:
def off_to_ply(data_root = 'ModelNet40', save_root = 'ply_file', folder = 'train'):
    
    #check if slash is added
    if save_root[-1] != "/":
        save_root = save_root + '/'
    if data_root[-1] != "/":
        data_root = data_root + '/'
        
    #create save root folder
    try:
        os.mkdir(save_root)
    except:
        pass
    
    #find all the categories
    cats  = os.listdir(data_root)
    if '.DS_Store' in cats:
        cats.remove('.DS_Store')
        
    for cat in cats:        
        
        #create direction from ply_root to ply_cat
        ply_root_to_cat =  save_root + f'{cat}/'
        try:
            os.mkdir(ply_root_to_cat)
        except:
            pass
        
        #create direction from ply_root to ply_group (train/test)
        ply_root_to_group = save_root + f'{cat}/' + f'{folder}/'
        try: 
            os.mkdir(ply_root_to_group)
        except:
            pass
        
        #define cad data path
        off_root_to_group = data_root + f'{cat}/' + f'{folder}/'
        off_file_list = os.listdir(off_root_to_group)
        if '.DS_Store' in off_file_list:
            file_path.remove('.DS_Store')
        off_file_list = [off_root_to_group + f for f in off_file_list]
        
        for off_file_path in off_file_list:
            with open(off_file_path, 'r') as f:
                #read until c_line is not 'OFF'
                str_line = f.readline().rstrip()
                if str_line != 'OFF':
                    str_line = str_line[3:]
                else:
                    str_line = f.readline().rstrip()
                str_line = str_line.split(' ')
                
                #get number of vertices and faces
                num_vertices = int(str_line[0])
                num_faces = int(str_line[1])
                
                #print(f"num_vertices: {num_vertices}, num_faces: {num_faces}")
                
                #get vertices data
                data_vertices = []
                for _ in range(num_vertices):
                    xyz = [float(data) for data in f.readline().rstrip().split(' ')]
                    data_vertices.append(xyz)
                data_vertices = np.array(data_vertices)

                #define ply file name
                slash_idx = off_file_path.rfind('/')
                ply_file  = ply_root_to_group + off_file_path[slash_idx+1:].replace('off', 'ply')
                
                #print(f'ply_file: {ply_file}')
                
                #write data to ply file
                with open(ply_file, 'w') as plyFile:
                    # write header
                    plyFile.write('ply\n')
                    plyFile.write('format ascii 1.0\n')
                    plyFile.write('element vertex ')
                    plyFile.write(str(num_vertices))
                    plyFile.write('\nproperty float32 x\n')
                    plyFile.write('property float32 y\n')
                    plyFile.write('property float32 z\n')
                    plyFile.write('element face ')
                    plyFile.write(str(num_faces))
                    plyFile.write('\nproperty list uint8 int32 vertex_indices\n')
                    plyFile.write('end_header\n')
                    
                    # Accum face text and collect vertex indices that actually used (avoid floating points)
                    face_strs = ''
                    val_idxs = set()
                    for _ in range(num_faces):
                        data_face = f.readline()
                        vals = [int(ind) for ind in data_face.rstrip().split(' ')]
                        for i in vals[1:]:
                            val_idxs.add(i)
                        face_strs += data_face

                    # Normalise vectices data (val_idxs)
                    filt_mean = np.mean(data_vertices[list(val_idxs), :], axis=0)
                    data_normed = data_vertices - filt_mean

                    # Compute the fartest point from the zero
                    max_dist = np.max(np.linalg.norm(data_normed[list(val_idxs), :], axis=1))
                    data_normed /= max_dist

                    # write vertices data to ply file
                    for pt in data_normed:
                        plyFile.write(' '.join(map(str, pt)))
                        plyFile.write('\n')

                    # write face data to ply file
                    plyFile.write(face_strs)

In [3]:
off_to_ply(folder = 'train')
off_to_ply(folder = 'test')

# Test Loss Functions

In [ ]:
y_pred = [[0, 1, 1.], [0.2, 0.2, 0.3]]
y_true = [[0.,1.,0.], [1.,0.,0.]]
y_pred = tf.constant(y_pred, dtype = tf.float32)
y_true = tf.constant(y_true)

print(custom_loss(y_true, y_pred))